In [1]:
import os

In [2]:
%pwd

'f:\\MLOps\\MLOps\\Kidney-Decease-Classification-End-to-End-MLflow-DVC\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\MLOps\\MLOps\\Kidney-Decease-Classification-End-to-End-MLflow-DVC'

In [5]:
MLFLOW_TRACKING_URI = 'https://dagshub.com/asadalipuh5/Kidney-Decease-Classification-End-to-End-MLflow-DVC.mlflow'
MLFLOW_TRACKING_USERNAME='asadalipuh5'
MLFLOW_TRACKING_PASSWORD='3db7b15bc073136a5cde57c45396d7feeffef8a3'

In [6]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/asadalipuh5/Kidney-Decease-Classification-End-to-End-MLflow-DVC.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='asadalipuh5'
os.environ['MLFLOW_TRACKING_PASSWORD']='3db7b15bc073136a5cde57c45396d7feeffef8a3'

In [38]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

## Entity

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## Configuration

In [40]:
from cnnClassifier.constants import *
from cnnClassifier.utils.utils import read_yaml, create_directories, save_json
from cnnClassifier.logger import logger
from cnnClassifier.exception import CustomException
import sys

In [41]:
class ConfigurationManager:
    
    """
    Manages the Configuration settings for the project.
    
    Reads the configurations form YAML files and sets up directories.
    """
    

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        """
        Initialized the ConfigurationManager.
        
        Args:
            config_filepath (str): Path to the configuration YAML file.
            params_filepath (str): Path to the paramters YAML file
        """
        
        try:
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
        
        except Exception as e:
            logger.error(f"Failed to read YAML files: {e}")
            raise CustomException(e, sys)
        
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/asadalipuh5/Kidney-Decease-Classification-End-to-End-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [42]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [43]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [44]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-15 05:23:11,317: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2024-10-15 05:23:11,334: INFO: utils: yaml file: params.yaml loaded successfully]
[2024-10-15 05:23:11,336: INFO: utils: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 44s 5s/step - loss: 3.5976 - accuracy: 0.6331
[2024-10-15 05:23:57,140: INFO: utils: json file saved at: scores.json]


2024/10/15 05:23:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-10-15 05:24:02,207: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\asada\AppData\Local\Temp\tmpe15x5790\model\data\model\assets
[2024-10-15 05:24:04,200: INFO: builder_impl: Assets written to: C:\Users\asada\AppData\Local\Temp\tmpe15x5790\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/15 05:25:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
